In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib as mpl

Using TensorFlow backend.


# 1）数据处理

In [2]:
df_doctorOrder = pd.read_excel('./patient_info.xls',sheet_name='Sheet1')
df_doctorOrder.head()

,患者住院ID,患者个人ID（标识符）,年龄,性别,职业,BMI,住院时长,症状首发年限,1年内急性加重次数,吸烟史,...,肺总量,残气量,残总比,慢支炎征象,肺气肿征象,肺动脉高压征象,肺部感染征象,纤维灶征象,支气管扩张征象,间质肺征象
0,ZY060000295315,295315,>65岁,男,农民,NaN,>7天~14天,COPD患病年限>=10~19年,无,NaN,...,NaN,NaN,NaN,NaN,有,NaN,NaN,NaN,NaN,NaN
1,ZY060000338471,338471,>65岁,男,个体经营者,NaN,<=7天,COPD患病年限<10年,无,NaN,...,NaN,NaN,NaN,NaN,有,NaN,NaN,NaN,NaN,NaN
2,ZY050000341147,341147,>55-65岁,男,个体经营者,NaN,>7天~14天,COPD患病年限<10年,无,无吸烟史,...,NaN,NaN,NaN,NaN,有,NaN,NaN,NaN,NaN,NaN
3,ZY020000329288,329288,>65岁,男,国家公务员,NaN,<=7天,COPD患病年限>=20年,无,NaN,...,NaN,NaN,NaN,NaN,有,NaN,NaN,NaN,NaN,NaN
4,ZY020000350407,350407,>65岁,男,个体经营者,NaN,>7天~14天,COPD患病年限>=20年,无,有吸烟史,...,NaN,NaN,NaN,NaN,有,NaN,NaN,NaN,NaN,NaN


In [3]:
df_doctorOrder_exact = df_doctorOrder.loc[:, ['PATIENT_ID','INPATIENT_NO','ITEM_CODE','MO_NAME','DATE_BGN','DATE_END','OPEN_TIME','ITEM_CLASS','normalized_item_code']]
df_doctorOrder_exact.head()

,PATIENT_ID,INPATIENT_NO,ITEM_CODE,MO_NAME,DATE_BGN,DATE_END,OPEN_TIME,ITEM_CLASS,normalized_item_code
0,0081041264,ZY070000581043,Y00000004991,善存片,2018/1/5 星期五 0:01:00,2018/1/12 星期五 10:42:19,2018/1/5 星期五 16:46:54,西药,XA11AA-XD185-01
1,0081265768,ZY010000595649,F00002227295,一次性使用输液器（超低密度聚乙烯精密输液器（带螺口））,2017/12/8 星期五 9:54:39,2017/12/8 星期五 10:25:05,2017/12/8 星期五 9:54:41,治疗,NaN
2,0080234445,ZY090000353163,18435,九项呼吸道感染病原体IgM抗体检测,2017/8/16 星期三 15:17:42,0001/1/1 星期一,2017/8/16 星期三 15:17:42,检验,18435
3,0000764987,ZY010000606770,F00002204394,动脉采血,2018/3/29 星期四 17:41:36,0001/1/1 星期一,2018/3/29 星期四 17:41:36,治疗,NaN
4,0000749211,ZY060000321757,F00002204391,静脉采血,2013/2/16 星期六 16:32:29,0001/1/1 星期一,2013/2/16 星期六 16:32:29,治疗,NaN


In [6]:
df_doctorOrderByID = df_doctorOrder_exact.groupby(['PATIENT_ID','INPATIENT_NO'])
print(df_doctorOrderByID.groups)

{('0000007630', 'ZY040000336335'): Int64Index([  1248,   1409,   3370,   5829,   9088,   9543,  10008,  10373,
             11185,  12699,
            ...
            292485, 292657, 294111, 294513, 296348, 297262, 298192, 300510,
            300632, 301088],
           dtype='int64', length=188), ('0000010623', 'ZY020000349626'): Int64Index([   955,  13801,  15518,  22314,  27356,  28175,  29323,  30070,
             34709,  35712,  36265,  38713,  38875,  42391,  43812,  45903,
             48489,  49647,  50647,  53829,  57795,  61862,  63583,  67622,
             82469,  89373,  90561,  96262, 100801, 101503, 102162, 102431,
            104948, 105611, 106061, 106782, 115900, 116210, 124548, 126300,
            128211, 131714, 134356, 134942, 149272, 160441, 161297, 162199,
            164721, 170413, 170635, 177946, 179276, 180823, 184385, 192311,
            192907, 193272, 197227, 197599, 197956, 199580, 199629, 199837,
            202438, 204422, 205477, 210118, 211284, 212518,

In [14]:
# 数据示例
df_doctorOrderByID.get_group(('0000014122', 'ZY070000138161'))

,PATIENT_ID,INPATIENT_NO,ITEM_CODE,MO_NAME,DATE_BGN,DATE_END,OPEN_TIME,ITEM_CLASS,normalized_item_code
3523,0000014122,ZY070000138161,Y00000018577,奥德金注射液,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:45:14,西药,XV06DX-XX140-02
7466,0000014122,ZY070000138161,Y00000019561,(益康宁)前列地尔注射液#,2015/8/5 星期三 0:01:00,2015/8/14 星期五 9:18:35,2015/8/5 星期三 11:05:25,西药,XC01EA-XQ015-02
14714,0000014122,ZY070000138161,17100,CX3生化八项,2015/8/4 星期二 10:42:19,0001/1/1 星期一,2015/8/4 星期二 10:42:19,检验,17100
15065,0000014122,ZY070000138161,999,明天补液后出院,2015/8/13 星期四 9:49:42,0001/1/1 星期一,2015/8/13 星期四 9:49:42,预约出院,999
16860,0000014122,ZY070000138161,F00002227295,一次性使用输液器（超低密度聚乙烯精密输液器（带螺口））,2015/8/4 星期二 11:22:36,2015/8/14 星期五 9:18:35,2015/8/4 星期二 11:22:37,治疗,NaN
21388,0000014122,ZY070000138161,17943,EB病毒抗体测定,2015/8/4 星期二 10:43:00,0001/1/1 星期一,2015/8/4 星期二 10:43:00,检验,17943
23959,0000014122,ZY070000138161,F00002204391,静脉采血,2015/8/4 星期二 10:42:20,0001/1/1 星期一,2015/8/4 星期二 10:42:20,治疗,NaN
25392,0000014122,ZY070000138161,Y00000018274,0.9%氯化钠注射液&#@,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:45:14,西药,XB05XA-XL211-02
31718,0000014122,ZY070000138161,Y00000018477,盖三淳胶丸#,2015/8/4 星期二 0:01:00,2015/8/14 星期五 9:18:35,2015/8/4 星期二 10:46:18,西药,XA11CC-XG111-01
32471,0000014122,ZY070000138161,16179,酶八项,2015/8/4 星期二 10:42:19,0001/1/1 星期一,2015/8/4 星期二 10:42:19,检验,16179


In [48]:
#  全部非重复的item_code数组
list_itemCode = df_doctorOrder_exact['ITEM_CODE'].unique().tolist()
count_uniqueItemCode = len(list_itemCode)
print(count_uniqueItemCode)

2536


In [30]:
#  病人最大访问次数
count_maxVisit = df_doctorOrder_exact.drop_duplicates('INPATIENT_NO').groupby('PATIENT_ID').size().max()
print(count_maxVisit)

15


In [35]:
#  病人数量
count_patientNum = df_doctorOrder_exact['PATIENT_ID'].unique().size
print(count_patientNum)

893


In [46]:
data_3D_itemCode = np.zeros((count_patientNum, count_maxVisit, count_uniqueItemCode))
print(data_3D_itemCode.shape) # （患者数量，单个患者最大访问次数，全部非重复编码数量）

(893, 15, 2536)


In [52]:
list_patientID = list()
for name,group in df_doctorOrderByID:
    current_patientID = name[0]
    if current_patientID not in list_patientID:
        list_patientID.append(current_patientID)
        list_visitOfCurrentPatient = list()
    index_patient = len(list_patientID)-1
    
    current_inpatientNO = name[1]
    list_visitOfCurrentPatient.append(current_inpatientNO)
    index_visitOfCurrentPatient = len(list_visitOfCurrentPatient)-1
    
    current_visit_codes = group['ITEM_CODE']
    for code in current_visit_codes:
        index_code = list_itemCode.index(code)
        data_3D_itemCode[index_patient][index_visitOfCurrentPatient][index_code] = 1
    
# 所有患者
print(data_3D_itemCode)

[[[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


# 2）针对患者单次访问的医学编码稀疏向量进行嵌入

In [57]:
data_2D_itemCode = np.array(data_3D_itemCode).reshape(-1, count_uniqueItemCode)
data_2D_itemCode.shape

(13395, 2536)

In [66]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

In [83]:
embedding_length = 128
inputs = keras.Input(shape=(count_uniqueItemCode,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = keras.layers.Dense(512, activation='relu')(inputs)
embedding = keras.layers.Dense(embedding_length, activation='relu')(x)
y = keras.layers.Dense(512, activation='relu')(embedding)
predictions = keras.layers.Dense(count_uniqueItemCode, activation='relu')(y)

# Instantiate the model given inputs and outputs.
model = keras.Model(inputs=inputs, outputs=predictions)

# The compile step specifies the training configuration.
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [84]:
tensorboard = TensorBoard(log_dir='log',
                    histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True,  # 是否可视化梯度直方图
                 write_images=True  # 是否可视化参数
                )
                          
checkpoint = ModelCheckpoint(filepath='patientVisit_embedding_model.h5',monitor='val_acc',mode='auto' ,save_best_only='True')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)

callback_lists=[checkpoint, tensorboard, early_stopping]

# Trains for 5 epochs
model.fit(data_2D_itemCode, data_2D_itemCode, batch_size=32, epochs=100, validation_split=0.2, callbacks=callback_lists)

Train on 10716 samples, validate on 2679 samples
Epoch 1/100
10716/10716 [==============================] - 9s 831us/step - loss: 0.0035 - acc: 0.0029 - val_loss: 0.0020 - val_acc: 0.0000e+00
Epoch 2/100
  192/10716 [..............................] - ETA: 8s - loss: 0.0028 - acc: 0.0000e+00

/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:118: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


10716/10716 [==============================] - 9s 855us/step - loss: 0.0024 - acc: 0.0220 - val_loss: 0.0018 - val_acc: 3.7327e-04
Epoch 3/100
10716/10716 [==============================] - 10s 954us/step - loss: 0.0021 - acc: 0.2125 - val_loss: 0.0016 - val_acc: 0.0000e+00
Epoch 4/100
10716/10716 [==============================] - 9s 805us/step - loss: 0.0019 - acc: 0.4560 - val_loss: 0.0016 - val_acc: 0.9227
Epoch 5/100
10716/10716 [==============================] - 10s 908us/step - loss: 0.0019 - acc: 0.6337 - val_loss: 0.0015 - val_acc: 0.9224
Epoch 6/100
10716/10716 [==============================] - 10s 959us/step - loss: 0.0018 - acc: 0.6869 - val_loss: 0.0015 - val_acc: 0.9220
Epoch 7/100
10716/10716 [==============================] - 10s 896us/step - loss: 0.0018 - acc: 0.7429 - val_loss: 0.0015 - val_acc: 0.9235
Epoch 8/100
10716/10716 [==============================] - 9s 875us/step - loss: 0.0017 - acc: 0.7903 - val_loss: 0.0015 - val_acc: 0.9227
Epoch 9/100
10716/10716 [==

In [85]:
del model
model = keras.models.load_model('patientVisit_embedding_model.h5')

/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [86]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 2536)              0         
_________________________________________________________________
dense_37 (Dense)             (None, 512)               1298944   
_________________________________________________________________
dense_38 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_39 (Dense)             (None, 512)               66048     
_________________________________________________________________
dense_40 (Dense)             (None, 2536)              1300968   
Total params: 2,731,624
Trainable params: 2,731,624
Non-trainable params: 0
_________________________________________________________________
None


In [88]:
embedding_layer_model = keras.Model(inputs=model.input,
                                     outputs=model.get_layer('dense_38').output)
#以这个model的预测值作为输出
embedding_output = embedding_layer_model.predict(data_2D_itemCode)
 
print(embedding_output.shape)
print(embedding_output[0])

(13395, 128)
[0.         0.22466484 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 2.0885463  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.45330447 0.         0.         0.         0.
 0.         0.13245502 0.         0.         0.         0.
 0.         0.         0.40092218 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.17747174 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.03939337
 0.         0.         0.         0.         0.         0.
 0.         1.2983484  0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0

In [89]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.01),
              loss='mean_squared_error',
              metrics=['accuracy'])
score = model.evaluate(data_2D_itemCode, data_2D_itemCode, verbose=0)
print(score)

[0.0014658887922798936, 0.8953340799117008]


# 3）应用CNN进一步提取患者相邻访问间局部重要特征，来表达患者纵向EHR

In [90]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

In [93]:
data_3D_itemCode_embedded = embedding_output.reshape((count_patientNum, count_maxVisit, embedding_length))
print(data_3D_itemCode_embedded.shape)

(893, 15, 128)


In [ ]:
model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=data_3D_itemCode_embedded.shape))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, epochs=10)
score = model.evaluate(data_3D_itemCode_embedded, data_3D_itemCode_embedded, batch_size=16)

In [122]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
from keras import backend as K

input_code = Input(shape=(count_maxVisit, embedding_length))  # adapt this if using `channels_first` image data format

x = Conv1D(16, 3, activation='relu', padding='same')(input_code)
x = MaxPooling1D(1, padding='same')(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = MaxPooling1D(5, padding='same')(x)
x = Conv1D(8,3, activation='relu', padding='same')(x)
encoded = MaxPooling1D(3, padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv1D(8, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(3)(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
x = UpSampling1D(5)(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
x = UpSampling1D(1)(x)
decoded = Conv1D(128, 3, activation='sigmoid', padding='same')(x)

autoencoder = Model(input_code, decoded)
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error', metrics=['accuracy'])

print(autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 15, 128)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 15, 16)            6160      
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 15, 16)            0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 15, 8)             392       
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 3, 8)              0         
_________________________________________________________________
conv1d_65 (Conv1D)           (None, 3, 8)              200       
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 1, 8)              0         
__________

In [124]:
tensorboard = TensorBoard(log_dir='./log/autoencoder',
                    histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True,  # 是否可视化梯度直方图
                 write_images=True  # 是否可视化参数
                )
                          
checkpoint = ModelCheckpoint(filepath='patientVisit_CNN_embedding_model.h5',monitor='val_acc',mode='auto' ,save_best_only='True')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)

callback_lists=[checkpoint, tensorboard, early_stopping]

autoencoder.fit(data_3D_itemCode_embedded, data_3D_itemCode_embedded,
                epochs=500,
                batch_size=128,
                shuffle=True,
                validation_split=0.2,
                callbacks=callback_lists)


Train on 714 samples, validate on 179 samples
Epoch 1/500
714/714 [==============================] - 0s 191us/step - loss: 0.0928 - acc: 0.0000e+00 - val_loss: 0.0765 - val_acc: 0.0000e+00
Epoch 2/500
714/714 [==============================] - 0s 178us/step - loss: 0.0927 - acc: 0.0000e+00 - val_loss: 0.0765 - val_acc: 0.0000e+00
Epoch 3/500
714/714 [==============================] - 0s 170us/step - loss: 0.0926 - acc: 0.0000e+00 - val_loss: 0.0764 - val_acc: 0.0000e+00
Epoch 4/500
714/714 [==============================] - 0s 177us/step - loss: 0.0926 - acc: 0.0000e+00 - val_loss: 0.0764 - val_acc: 0.0000e+00
Epoch 5/500
714/714 [==============================] - 0s 174us/step - loss: 0.0926 - acc: 0.0000e+00 - val_loss: 0.0764 - val_acc: 0.0000e+00
Epoch 6/500
714/714 [==============================] - 0s 178us/step - loss: 0.0925 - acc: 0.0000e+00 - val_loss: 0.0763 - val_acc: 0.0000e+00
Epoch 7/500
714/714 [==============================] - 0s 194us/step - loss: 0.0925 - acc: 0.000

In [125]:
from keras.models import load_model
from keras.layers import *
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import preprocessing

In [127]:
data_2D_itemCode_by_patient = data_3D_itemCode_embedded.reshape(count_patientNum, -1)
print(data_2D_itemCode_by_patient.shape)

(893, 1920)


In [130]:
matrix_patient_similarity = cosine_similarity(data_2D_itemCode_by_patient)
print(matrix_patient_similarity)

[[0.9999999  0.80015904 0.5699328  ... 0.96690786 0.9623244  0.98765254]
 [0.80015904 0.99999976 0.5499156  ... 0.7994744  0.7874751  0.8070792 ]
 [0.5699328  0.5499156  1.         ... 0.5624945  0.5586596  0.57381785]
 ...
 [0.96690786 0.7994744  0.5624945  ... 0.99999976 0.9921602  0.9828981 ]
 [0.9623244  0.7874751  0.5586596  ... 0.9921602  0.9999999  0.97769034]
 [0.98765254 0.8070792  0.57381785 ... 0.9828981  0.97769034 0.9999999 ]]


In [134]:
list_mostSimilairPatient = list()
for i in range(len(matrix_patient_similarity)):    
    list_currentPatientSimilairP = matrix_patient_similarity[i].tolist()
    del list_currentPatientSimilairP[i]
    max_index = list_currentPatientSimilairP.index(max(list_currentPatientSimilairP))
    max_similarity = list_currentPatientSimilairP[max_index]
    
    list_currentPatientSimilairP_origin = matrix_patient_similarity[i].tolist()
    for ii in range(len(list_currentPatientSimilairP_origin)):
        if list_currentPatientSimilairP_origin[ii] == max_similarity and ii != i:
            list_mostSimilairPatient.append(ii)
            break
        
print(len(list_mostSimilairPatient))   

893


In [135]:
selected_current_patientID = list_patientID[0]
selected_current_mostSimilar_patientID = list_patientID[list_mostSimilairPatient[0]]
print(selected_current_patientID)
print(selected_current_mostSimilar_patientID)

0000007630
0081344080


In [136]:
df_doctorOrderByPatientID = df_doctorOrder_exact.groupby('PATIENT_ID')
print(df_doctorOrderByPatientID.groups)

{'0000007630': Int64Index([  1248,   1409,   3370,   5829,   9088,   9543,  10008,  10373,
             11185,  12699,
            ...
            292485, 292657, 294111, 294513, 296348, 297262, 298192, 300510,
            300632, 301088],
           dtype='int64', length=188), '0000010623': Int64Index([   790,    906,    955,   1811,   2906,   2971,   3492,   3824,
              5714,   6632,
            ...
            295430, 295893, 296277, 296721, 298226, 298302, 299063, 299489,
            300486, 301108],
           dtype='int64', length=560), '0000014122': Int64Index([   771,   1669,   2345,   2396,   3356,   3523,   4361,   4383,
              4721,   5000,
            ...
            296678, 297420, 297658, 298267, 299256, 299677, 300491, 300814,
            300903, 301163],
           dtype='int64', length=619), '0000022807': Int64Index([  1005,   1581,   2246,   4198,   8966,   9321,  11661,  15613,
             23731,  28518,
            ...
            281658, 281811, 282

In [146]:
selected_current_patientData =  df_doctorOrderByPatientID.get_group(selected_current_patientID)
print(selected_current_patientData.shape)
print(selected_current_patientData)

(188, 9)
        PATIENT_ID    INPATIENT_NO     ITEM_CODE                      MO_NAME  \
1248    0000007630  ZY040000336335  Y00000020121                0.9%氯化钠注射液&#@   
1409    0000007630  ZY040000336335           999                 测BP、P、R、SPO2   
3370    0000007630  ZY040000336335  Y00000019982                      明可欣针**#   
5829    0000007630  ZY040000336335  F00002204397            静脉输液(连续输液第二组起每组收)   
9088    0000007630  ZY040000336335  F00002218596               一次性注射器30ML/因特圣   
9543    0000007630  ZY040000336335  Y00000001533                     消脱止-M片**   
10008   0000007630  ZY040000336335  F00002218595               一次性注射器20ML/因特圣   
10373   0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
11185   0000007630  ZY040000336335  Y00000000673                 葡萄糖氯化钠注射液&#@   
12699   0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
15535   0000007630  ZY040000336335  F00002204346                         院内会诊   
16464   0000007630 

In [147]:
selected_current_mostSimilar_patientData = df_doctorOrderByPatientID.get_group(selected_current_mostSimilar_patientID)
print(selected_current_mostSimilar_patientData.shape)
print(selected_current_mostSimilar_patientData)

(158, 9)
        PATIENT_ID    INPATIENT_NO     ITEM_CODE                      MO_NAME  \
3976    0081344080  ZY010000608852           999                           普食   
5647    0081344080  ZY010000608852  F00002213154                        引流管引流   
7340    0081344080  ZY010000608852  Y00000018919                  地佐辛注射液(加罗宁)   
7815    0081344080  ZY010000608852  F00002218595               一次性注射器20ML/因特圣   
8503    0081344080  ZY010000608852  Y00000019941                       阿托品注射液   
10032   0081344080  ZY010000608852  F00002212422                        黄头真空管   
12727   0081344080  ZY010000608852  F00002204400                       住院静脉输液   
15765   0081344080  ZY010000608852  Y00000020216            0.9%氯化钠注射液(科伦)&#@   
20139   0081344080  ZY010000608852  F00002212422                        黄头真空管   
22066   0081344080  ZY010000608852  F00002204352                         Ⅰ级护理   
24396   0081344080  ZY010000608852           999                        午夜后禁食   
35604   0081344080 

In [157]:
df_compareSimilar = pd.merge(selected_current_patientData[['PATIENT_ID','INPATIENT_NO','ITEM_CODE','MO_NAME','ITEM_CLASS','normalized_item_code']], 
                             selected_current_mostSimilar_patientData[['PATIENT_ID','INPATIENT_NO','ITEM_CODE','MO_NAME','ITEM_CLASS','normalized_item_code']], 
                             how='outer', on=['ITEM_CODE', 'MO_NAME', 'ITEM_CLASS', 'normalized_item_code'])
print(df_compareSimilar.head())
print(df_compareSimilar.shape)

  PATIENT_ID_x  INPATIENT_NO_x     ITEM_CODE        MO_NAME ITEM_CLASS  \
0   0000007630  ZY040000336335  Y00000020121  0.9%氯化钠注射液&#@         西药   
1   0000007630  ZY040000336335  Y00000020121  0.9%氯化钠注射液&#@         西药   
2   0000007630  ZY040000336335           999   测BP、P、R、SPO2         护理   
3   0000007630  ZY040000336335  Y00000019982        明可欣针**#         西药   
4   0000007630  ZY040000336335  Y00000019982        明可欣针**#         西药   

  normalized_item_code PATIENT_ID_y  INPATIENT_NO_y  
0      XB05XA-XL211-02          NaN             NaN  
1      XB05XA-XL211-02          NaN             NaN  
2                  999          NaN             NaN  
3      XJ01DC-XT070-02   0081344080  ZY010000608852  
4      XJ01DC-XT070-02   0081344080  ZY010000608852  
(467, 8)


In [158]:
df_compareSimilar_dropDuplication = df_compareSimilar.drop_duplicates(['ITEM_CODE','MO_NAME'])
print(df_compareSimilar_dropDuplication.shape)
print(df_compareSimilar_dropDuplication)

(170, 8)
    PATIENT_ID_x  INPATIENT_NO_x     ITEM_CODE                      MO_NAME  \
0     0000007630  ZY040000336335  Y00000020121                0.9%氯化钠注射液&#@   
2     0000007630  ZY040000336335           999                 测BP、P、R、SPO2   
3     0000007630  ZY040000336335  Y00000019982                      明可欣针**#   
7     0000007630  ZY040000336335  F00002204397            静脉输液(连续输液第二组起每组收)   
37    0000007630  ZY040000336335  F00002218596               一次性注射器30ML/因特圣   
41    0000007630  ZY040000336335  Y00000001533                     消脱止-M片**   
42    0000007630  ZY040000336335  F00002218595               一次性注射器20ML/因特圣   
96    0000007630  ZY040000336335  Y00000000469                0.9%氯化钠注射液&#@   
111   0000007630  ZY040000336335  Y00000000673                 葡萄糖氯化钠注射液&#@   
114   0000007630  ZY040000336335  F00002204346                         院内会诊   
117   0000007630  ZY040000336335  Y00000002188                     可必特溶液**#   
118   0000007630  ZY040000336335  F00000181

In [159]:
df_compareSimilar_dropDuplication.to_csv('CompareSimilarPatient01.csv')